In [4]:
%tensorflow_version 2.x
import os
from shutil import rmtree
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.48.30.82:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.48.30.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.48.30.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
max_len = 500
top_words = 5000
max_words = 10000
path_to_data = './'
glove_dir = './'
embedding_dim = 300
embedding_file_name = 'glove.6B.300d.txt'

!wget -O glove6b.zip http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove6b.zip

--2021-05-23 04:01:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-23 04:01:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-23 04:01:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove6b.zip’

glove6b.

In [29]:
emotion_data = pd.read_csv("emotion_data_bin.csv")
#empathy_data = pd.read_csv("empaty_assessment.csv")
emotion_data = emotion_data[["utterance", "emotion"]]
emotion_data.head()

,utterance,emotion
0,I remember going to see the fireworks with my ...,1
1,This was a best friend. I miss her.,1
2,We no longer talk.,1
3,it feels like hitting to blank wall when i se...,1
4,dont you feel so.. its a wonder,1


In [30]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(emotion_data["utterance"], emotion_data["context"], test_size=0.33, random_state=42, stratify=emotion_data["context"])

labels = emotion_data["emotion"]#pd.get_dummies(emotion_data['emotion']).values
texts = emotion_data.utterance
data = emotion_data

In [31]:
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)   

In [32]:
X_train = data
Y_train = labels

In [33]:
# create dictionary to map word -> embedding vector
embeddings_index = {}                                        
f = open(os.path.join(glove_dir, embedding_file_name))
i = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print("Found %s word vectors." % len(embeddings_index)) 

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [34]:
embedding_matrix = preprocessing.scale(embedding_matrix)

In [35]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    beta_sqr = (1.2**2)
    return (1 + beta_sqr) *((precision*recall)/((beta_sqr * precision)+recall+K.epsilon()))

In [36]:
def create_model(max_words, embedding_dim, max_len, embedding_matrix):
    hidden_dim = 300 
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(Bidirectional(LSTM(hidden_dim)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))# Dense(32, activation='softmax'))
    
    # Load embeddings
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False   
    
    # Compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m])#categorical_crossentropy
    
    return model

In [37]:
# Enter TPU env:
with tpu_strategy.scope():
    # instantiate model
    model = create_model(max_words, embedding_dim, max_len, embedding_matrix)     
    # train model
    history = model.fit(X_train, Y_train, epochs = 15)

Epoch 1/15
1774/1774 [==============================] - 141s 76ms/step - loss: 0.2710 - acc: 0.8780 - f1_m: 0.9092
Epoch 2/15
1774/1774 [==============================] - 134s 76ms/step - loss: 0.1680 - acc: 0.9301 - f1_m: 0.9446
Epoch 3/15
1774/1774 [==============================] - 134s 76ms/step - loss: 0.1248 - acc: 0.9504 - f1_m: 0.9615
Epoch 4/15
1774/1774 [==============================] - 135s 76ms/step - loss: 0.0903 - acc: 0.9637 - f1_m: 0.9693
Epoch 5/15
1774/1774 [==============================] - 135s 76ms/step - loss: 0.0638 - acc: 0.9751 - f1_m: 0.9776
Epoch 6/15
1774/1774 [==============================] - 135s 76ms/step - loss: 0.0498 - acc: 0.9807 - f1_m: 0.9830
Epoch 7/15
1774/1774 [==============================] - 134s 76ms/step - loss: 0.0400 - acc: 0.9855 - f1_m: 0.9863
Epoch 8/15
1774/1774 [==============================] - 134s 76ms/step - loss: 0.0330 - acc: 0.9875 - f1_m: 0.9888
Epoch 9/15
1774/1774 [==============================] - 135s 76ms/step - loss: 0

In [38]:
valid = pd.read_csv("emotion_data_bin_valid.csv")

labels = valid["emotion"]#pd.get_dummies(valid['context']).values
texts = valid.utterance
data = valid
sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)   
X_test = data
Y_test = labels

In [48]:
# Enter TPU env:
with tpu_strategy.scope():
    # print fold classification report
    y_hat = model.predict_classes(X_test, verbose = 0)
    y_hat_classes = [y[0] for y in y_hat]
    y_test_classes = Y_test

print(classification_report(y_test_classes, y_hat_classes, digits=4))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0     0.8339    0.8229    0.8283      1592
           1     0.9557    0.9588    0.9572      6340

    accuracy                         0.9315      7932
   macro avg     0.8948    0.8908    0.8928      7932
weighted avg     0.9312    0.9315    0.9314      7932



In [49]:
precision_recall_fscore_support(y_test_classes,y_hat_classes, beta=1.2)[2]

array([0.82733701, 0.95753296])

In [50]:
model.save('BiLSTM_emotion_detection.h5')



---



In [51]:
emotion_data = pd.read_csv("emotion_data_class.csv")

In [52]:
labels = pd.get_dummies(emotion_data['context']).values
texts = emotion_data.utterance
data = emotion_data

In [53]:
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)

In [54]:
X_train = data
Y_train = labels

In [58]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    beta_sqr = 1
    return (1 + beta_sqr) *((precision*recall)/((beta_sqr * precision)+recall+K.epsilon()))

In [59]:
def create_model(max_words, embedding_dim, max_len, embedding_matrix):
    hidden_dim = 300 
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(Bidirectional(LSTM(hidden_dim)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='softmax'))# Dense(32, activation='softmax'))
    
    # Load embeddings
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False   
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', f1_m])#categorical_crossentropy
    
    return model

In [60]:
# Enter TPU env:
with tpu_strategy.scope():
    # instantiate model
    model = create_model(max_words, embedding_dim, max_len, embedding_matrix)     
    # train model
    history = model.fit(X_train, Y_train, epochs = 15)

Epoch 1/15
1373/1373 [==============================] - 111s 76ms/step - loss: 3.3414 - acc: 0.0886 - f1_m: 0.0037
Epoch 2/15
1373/1373 [==============================] - 103s 75ms/step - loss: 2.7324 - acc: 0.2426 - f1_m: 0.0820
Epoch 3/15
1373/1373 [==============================] - 103s 75ms/step - loss: 2.3954 - acc: 0.3239 - f1_m: 0.1792
Epoch 4/15
1373/1373 [==============================] - 103s 75ms/step - loss: 2.1575 - acc: 0.3801 - f1_m: 0.2490
Epoch 5/15
1373/1373 [==============================] - 104s 76ms/step - loss: 1.9341 - acc: 0.4381 - f1_m: 0.3182
Epoch 6/15
1373/1373 [==============================] - 103s 75ms/step - loss: 1.7298 - acc: 0.4890 - f1_m: 0.3884
Epoch 7/15
1373/1373 [==============================] - 104s 76ms/step - loss: 1.5844 - acc: 0.5267 - f1_m: 0.4386
Epoch 8/15
1373/1373 [==============================] - 103s 75ms/step - loss: 1.4205 - acc: 0.5718 - f1_m: 0.5030
Epoch 9/15
1373/1373 [==============================] - 104s 75ms/step - loss: 1

In [61]:
valid = pd.read_csv("emotion_data_class_valid.csv")

labels = pd.get_dummies(valid['context']).values
texts = valid.utterance
data = valid
sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)   
X_test = data
Y_test = labels

In [62]:
# Enter TPU env:
with tpu_strategy.scope():
    # print fold classification report
    y_hat = model.predict(X_test, verbose = 0)
    y_hat_classes = tf.argmax(y_hat, axis=1).numpy()
    y_test_classes = tf.argmax(Y_test, axis=1).numpy()

print(classification_report(y_test_classes, y_hat_classes, digits=4))

              precision    recall  f1-score   support

           0     0.1828    0.1635    0.1726       208
           1     0.1173    0.1173    0.1173       196
           2     0.2564    0.2128    0.2326       282
           3     0.1987    0.1813    0.1896       171
           4     0.1630    0.2022    0.1805       183
           5     0.1257    0.1250    0.1254       176
           6     0.1583    0.1341    0.1452       164
           7     0.2088    0.2275    0.2178       167
           8     0.2659    0.3349    0.2965       212
           9     0.2615    0.2864    0.2734       199
          10     0.1782    0.1771    0.1777       175
          11     0.1137    0.1116    0.1127       215
          12     0.3907    0.3155    0.3491       187
          13     0.3558    0.3394    0.3474       218
          14     0.2115    0.2133    0.2124       225
          15     0.1688    0.1140    0.1361       114
          16     0.1961    0.1676    0.1807       179
          17     0.1694    

In [63]:
model.save('BiLSTM_emotion_class.h5')



---



In [107]:
emotion_data = pd.read_csv("empathy_data_assessment.csv")
emotion_data.head()

,context_utterance,min_empathy_eval,min_relevance_eval,min_understandability_eval
0,sentimental - Was this a friend you were in lo...,2,2,5
1,sentimental - Where has she gone?,2,2,5
2,sentimental - Oh was this something that happe...,2,2,5
3,afraid - Oh ya? I don't really see how,3,3,4
4,afraid - I do actually hit blank walls a lot o...,3,3,4


In [116]:
labels = pd.get_dummies(emotion_data["min_empathy_eval"]).values
texts = emotion_data["context_utterance"]
data = emotion_data

In [111]:
emotion_data = emotion_data[emotion_data["min_empathy_eval"] != 0]

In [117]:
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
eval = np.asarray(eval)

In [118]:
X_train = data
Y_train = labels

In [119]:
def create_model(max_words, embedding_dim, max_len, embedding_matrix):
    hidden_dim = 300 
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(Bidirectional(LSTM(hidden_dim)))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation="softmax"))
    
    # Load embeddings
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False   
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', f1_m])
    
    return model

In [120]:
# Enter TPU env:
with tpu_strategy.scope():
    # instantiate model
    model = create_model(max_words, embedding_dim, max_len, embedding_matrix)     
    # train model
    history = model.fit(X_train, Y_train, epochs = 14)

Epoch 1/14
1240/1240 [==============================] - 102s 77ms/step - loss: 1.0023 - acc: 0.6459 - f1_m: 0.6279
Epoch 2/14
1240/1240 [==============================] - 93s 75ms/step - loss: 0.9664 - acc: 0.6494 - f1_m: 0.6420
Epoch 3/14
1240/1240 [==============================] - 94s 75ms/step - loss: 0.9496 - acc: 0.6502 - f1_m: 0.6378
Epoch 4/14
1240/1240 [==============================] - 94s 76ms/step - loss: 0.9066 - acc: 0.6573 - f1_m: 0.6401
Epoch 5/14
1240/1240 [==============================] - 94s 76ms/step - loss: 0.8662 - acc: 0.6674 - f1_m: 0.6473
Epoch 6/14
1240/1240 [==============================] - 94s 76ms/step - loss: 0.8147 - acc: 0.6810 - f1_m: 0.6577
Epoch 7/14
1240/1240 [==============================] - 94s 75ms/step - loss: 0.7342 - acc: 0.7144 - f1_m: 0.6921
Epoch 8/14
1240/1240 [==============================] - 93s 75ms/step - loss: 0.6688 - acc: 0.7357 - f1_m: 0.7204
Epoch 9/14
1240/1240 [==============================] - 94s 76ms/step - loss: 0.6044 - 

In [121]:
valid = pd.read_csv("empathy_data_assessment_valid.csv")

labels = pd.get_dummies(valid["min_empathy_eval"]).values
texts = valid["context_utterance"]
data = valid

sequences = tokenizer.texts_to_sequences(texts)                
word_index = tokenizer.word_index   
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)   
X_test = data
Y_test = labels

In [103]:
valid["min_empathy_eval"].value_counts()

5    1972
4    1063
3     245
2      65
1      56
Name: min_empathy_eval, dtype: int64

In [106]:
Y_train

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]], dtype=uint8)

In [122]:
# Enter TPU env:
with tpu_strategy.scope():
    # print fold classification report
    y_hat = model.predict(X_test, verbose = 0)
    y_hat_classes = tf.argmax(y_hat, axis=1).numpy()
    y_test_classes = tf.argmax(Y_test, axis=1).numpy()

print(classification_report(y_test_classes, y_hat_classes, digits=4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        56
           1     0.0769    0.0154    0.0256        65
           2     0.1400    0.0571    0.0812       245
           3     0.3591    0.1966    0.2541      1063
           4     0.5932    0.8119    0.6855      1972

    accuracy                         0.5366      3401
   macro avg     0.2338    0.2162    0.2093      3401
weighted avg     0.4677    0.5366    0.4832      3401

